In [ ]:
from src.hpe.mp.performance import EstimationCollector

collector = EstimationCollector()
df_mp = collector.collect(name="estimations")

In [ ]:
from src.hpe.yolo.performance import EstimationCollector as YoloEstimationCollector

collector = YoloEstimationCollector()
df_yolo = collector.collect(name="estimations")

In [1]:
from src.hpe.mp.performance import read_estimations as read_mp_estimations
from src.hpe.yolo.performance import read_estimations as read_yolo_estimations

df_mp = read_mp_estimations()
df_yolo = read_yolo_estimations()

2025-10-10 09:31:08.253931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760081469.305018  990567 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760081469.647917  990567 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-10 09:31:12.311226: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


loaded labels


In [3]:

from src.hpe.common.typing import HpeEstimation
from pandas import Series

prediction_results = df_mp.map(HpeEstimation.prediction_result)
counts = prediction_results.apply(Series.value_counts).fillna(0)

In [6]:
from math import nan
from pandas import DataFrame
from src.common.helpers import safe_index

def _precision(totals: DataFrame, verbose: bool = False) -> float:
    if totals[""] > 0:
        return nan
    
    tp = safe_index(totals, "TP")
    fp = safe_index(totals, "FP")

    if verbose:
        print(f"TP: {tp}")
        print(f"FP: {fp}")

    if tp + fp == 0:
        return 0.0
    
    result = tp  / (tp + fp)
    if verbose:
        print(f"{tp} / ({tp} + {fp}) = {result}")

    return result

def _recall(totals: DataFrame, verbose: bool = False) -> float:
    if totals[""] > 0:
        return nan
    
    tp = safe_index(totals, "TP")
    fn = safe_index(totals, "FN")

    if verbose:
        print(f"TP: {tp}")
        print(f"FN: {fn}")

    if tp + fn == 0:
        return 0
    
    result = tp  / (tp + fn)
    if verbose:
        print(f"{tp} / ({tp} + {fn}) = {result}")
    return result

In [7]:

precisions = counts.apply(_precision, axis=0)
recalls = counts.apply(_recall, axis=0)

In [ ]:
from src.hpe.common.typing import HpeEstimation

df_mp_true = df_mp.map(lambda x: x.true_landmark)

In [ ]:
fp_pred = df_mp.iloc[9]['HEAD']
fp_pred.prediction_result(verbose=True)

In [ ]:
fp_pred = df_yolo.iloc[9]['HEAD']
fp_pred.prediction_result(verbose=True)

In [ ]:
from src.hpe.common.metrics import calc_precision_and_recall

prec, rec = calc_precision_and_recall(df_mp)

In [ ]:
from src.hpe.common.plot import plot_precision_recall_curve
plot_precision_recall_curve(prec, rec)

In [ ]:
from src.hpe.yolo.performance import read_distances as read_yolo_distances
from src.hpe.mp.performance import read_distances as read_mp_distances
from src.hpe.common.plot import plot_distances_boxplot

mp_dist = read_mp_distances(dataset_name="distances")
mp_bgr_dist = read_mp_distances(dataset_name="distances_bgr")

yolo_dist = read_yolo_distances(dataset_name="distances")
yolo_bgr_dist = read_yolo_distances(dataset_name="distances_bgr")

In [ ]:
plot_distances_boxplot(
    None,
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))

In [ ]:
plot_distances_boxplot(
    (0, 10),
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))

In [ ]:
from src.hpe_dnn.kfold import HpeDnnFoldCrossValidation
from src.sota.kfold import SOTAFoldCrossValidation
from src.common.plot import box_plot_accuracies

box_plot_accuracies(
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-unbalanced-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-dr0.3-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-yolo-kf"),
    # HpeDnnFoldCrossValidation.evaluation_instance("arch2-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch3-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch4-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch5-kf"),
    SOTAFoldCrossValidation.evaluation_instance("yolo11n-kf", "yolov11n-cls"))